In [1]:
import torch
import torch.optim as optim
from torch_geometric.data import InMemoryDataset
from dataset import load_split
import pandas as pd
from torch_geometric.nn import TransE
from torch_geometric.nn.kge.loader import KGTripletLoader
from dataset import T

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"; "using " + device

'using cuda'

In [3]:
# # from torch_geometric.data import InMemoryDataset
# # import pandas as pd

# class PygLncTarD(InMemoryDataset):
#   """
#   pytorch geometric compatible version of LncTarD dataset.
  
#   NB: The class is not (yet) capable of downloading the dataset itself.
#     Instead, it expects the dataset to be pre-split and put inside the 
#     `data/` subdirectory such that it can be loaded with `load_split`.
#   """
  
#   def __init__(self, split_name, transform=None, device=None, verbose=1):
#     super().__init__(".", transform)
    
#     # load data
#     self.split_name = split_name
#     self.df = load_split(split_name)
#     # use full_df to obtain mappings, to ensure that all possible heads, 
#     # targets, and relations are available.
#     self.full_df = load_split("full")
    
#     # load mappings for one hot encoding from original dataset, such that
#     # all possible entities (=heads or tails) and relations are available.
#     # NB: pyg does not distinguish between head and tail entities, therefore
#     # they are concatenated to a large list of entities here
#     full_df = load_split("full")
#     entities = pd.concat([full_df["head"], full_df["tail"]], ignore_index=True).drop_duplicates(ignore_index=True)
#     self.ent2idx = pd.get_dummies(entities,dtype=float).idxmax()
#     self.rel2idx = pd.get_dummies(full_df["relation"].drop_duplicates(ignore_index=True),dtype=float).idxmax()
#     self.idx2ent, self.idx2rel = self.ent2idx.T, self.rel2idx.T
    
#     # NB: this is not a very efficient way of obtaining the dataset, but it
#     # is sufficient for a small dataset like LncTarD.
#     self.tuples = torch.empty((len(self.df), 3), dtype=torch.long, device=device)
#     num_failed = 0
#     for tup in self.df.itertuples(index=True):
#       # TODO: figure out how we can ignore the weird string encodings
#       try:
#         h, r, t = self.ent2idx[tup.head], self.rel2idx[tup.relation], self.ent2idx[tup.tail]
#         assert tup.Index < len(self.tuples), "tuple index out of range"
#         self.tuples[tup.Index] = torch.tensor([h,r,t])
#       except KeyError:
#         num_failed += 1
#     if verbose:
#       print("cannot load %d/%d tuple(s) from %s because of weird string encoding" % 
#             (num_failed, num_failed + len(self.tuples), split_name))

#   def len(self):
#     return len(self.tuples)
  
#   def get(self, idx):
#     return self.tuples[idx]
  
#   @property
#   def num_nodes(self):
#     """Return the number of node types available in this dataset."""
#     # NB: in https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/kge/base.html#KGEModel
#     # it looks like that this should return the # of available node TYPES.
#     return len(self.ent2idx)
  
#   @property
#   def num_edge_types(self):
#     """Return the number of edge types available in this dataset."""
#     return len(self.rel2idx)
  
#   @property
#   def edge_index(self):
#     """Return the edge indices of this dataset, as a tensor of (h,t) tuples."""
#     return self.tuples[:,[0,2]]
  
#   @property
#   def edge_type(self):
#     """Return the edge types as a tensor."""
#     return self.tuples[:,1]

In [4]:
train_data, val_data, test_data = (
  PygLncTarD("train", device=device),
  PygLncTarD("val", device=device),
  PygLncTarD("test", device=device),
)

cannot load 18/5437 tuple(s) from train because of weird string encoding
cannot load 1/678 tuple(s) from val because of weird string encoding
cannot load 2/679 tuple(s) from test because of weird string encoding


In [6]:
model = TransE(
  num_nodes=train_data.num_nodes,
  num_relations=train_data.num_edge_types,
  hidden_channels=50,
).to(device)

In [7]:
model

TransE(3367, num_relations=7, hidden_channels=50)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=.01)

In [9]:
loader = KGTripletLoader(
  *train_data.tuples.T, batch_size=1000, shuffle=True,
)

In [10]:
def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )

In [11]:
for epoch in range(1, 501):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, hits = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val Hits@10: {hits:.4f}')

Epoch: 001, Loss: 0.9849
Epoch: 002, Loss: 0.9479
Epoch: 003, Loss: 0.9056
Epoch: 004, Loss: 0.8706
Epoch: 005, Loss: 0.8375
Epoch: 006, Loss: 0.7926
Epoch: 007, Loss: 0.7539
Epoch: 008, Loss: 0.7114
Epoch: 009, Loss: 0.6622
Epoch: 010, Loss: 0.6241
Epoch: 011, Loss: 0.5885
Epoch: 012, Loss: 0.5568
Epoch: 013, Loss: 0.5395
Epoch: 014, Loss: 0.5008
Epoch: 015, Loss: 0.4836
Epoch: 016, Loss: 0.4658
Epoch: 017, Loss: 0.4456
Epoch: 018, Loss: 0.4360
Epoch: 019, Loss: 0.4184
Epoch: 020, Loss: 0.4035
Epoch: 021, Loss: 0.3956
Epoch: 022, Loss: 0.3889
Epoch: 023, Loss: 0.3827
Epoch: 024, Loss: 0.3714
Epoch: 025, Loss: 0.3627


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1820.84it/s]

Epoch: 025, Val Mean Rank: 1734.00, Val Hits@10: 0.0000
Epoch: 026, Loss: 0.3529
Epoch: 027, Loss: 0.3493
Epoch: 028, Loss: 0.3426
Epoch: 029, Loss: 0.3453
Epoch: 030, Loss: 0.3264
Epoch: 031, Loss: 0.3248
Epoch: 032, Loss: 0.3248
Epoch: 033, Loss: 0.3180
Epoch: 034, Loss: 0.3099
Epoch: 035, Loss: 0.3105
Epoch: 036, Loss: 0.3048
Epoch: 037, Loss: 0.3040
Epoch: 038, Loss: 0.3078
Epoch: 039, Loss: 0.2941
Epoch: 040, Loss: 0.2821
Epoch: 041, Loss: 0.3014


Epoch: 042, Loss: 0.2947
Epoch: 043, Loss: 0.2905
Epoch: 044, Loss: 0.2775
Epoch: 045, Loss: 0.2748
Epoch: 046, Loss: 0.2814
Epoch: 047, Loss: 0.2753
Epoch: 048, Loss: 0.2700
Epoch: 049, Loss: 0.2745
Epoch: 050, Loss: 0.2677


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2623.08it/s]

...

100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2950.62it/s]

Epoch: 475, Val Mean Rank: 1614.00, Val Hits@10: 0.0000
Epoch: 476, Loss: 0.1052
Epoch: 477, Loss: 0.1069
Epoch: 478, Loss: 0.1091
Epoch: 479, Loss: 0.1039
Epoch: 480, Loss: 0.1112
Epoch: 481, Loss: 0.1043
Epoch: 482, Loss: 0.1065
Epoch: 483, Loss: 0.1032
Epoch: 484, Loss: 0.1133
Epoch: 485, Loss: 0.1027
Epoch: 486, Loss: 0.1090
Epoch: 487, Loss: 0.1085
Epoch: 488, Loss: 0.1059
Epoch: 489, Loss: 0.1062
Epoch: 490, Loss: 0.1111
Epoch: 491, Loss: 0.1117
Epoch: 492, Loss: 0.1045
Epoch: 493, Loss: 0.1057


Epoch: 494, Loss: 0.1053
Epoch: 495, Loss: 0.1038
Epoch: 496, Loss: 0.0994
Epoch: 497, Loss: 0.1029
Epoch: 498, Loss: 0.1057
Epoch: 499, Loss: 0.1086
Epoch: 500, Loss: 0.1050


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2484.04it/s]

Epoch: 500, Val Mean Rank: 1586.50, Val Hits@10: 0.0000
